In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils import data
from collections import namedtuple
from IPython.display import Image
import time
%matplotlib inline
np.random.seed(2020)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cuda = torch.cuda.is_available()

In [ ]:
#Hyperparameter for tuining(Only MLP)
hyper = {
        "nEpochs":5,
        "lr":0.1,
        "kcontext":12,
        "lr_decayRate":0.0,
        "randomSeed":2021,
        "batchSize":256,
        "dataPath":'/content/gdrive/MyDrive/Frame-Level_Classification_of_Speech/',
        "weightDirName": './checkpoint/',
        "hiddenDims": [2048,1024,1024,512],
        "checkpointPath":"./checkpoint/model_5.txt"
    }

In [ ]:
train_data = (np.load(hyper['dataPath']+"/train.npy", allow_pickle=True), 
        np.load(hyper['dataPath']+"/train_labels.npy", allow_pickle=True))
dev_data = (np.load(hyper['dataPath']+"/dev.npy", allow_pickle=True), 
        np.load(hyper['dataPath']+"/dev_labels.npy", allow_pickle=True))
test_data = (np.load(hyper['dataPath']+"/test.npy", allow_pickle=True), 
        None)

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, dataset, k):
        self.k = k
        self.dataX = dataset[0]
        self.dataY = dataset[1] if len(dataset) == 2 else None
        self.idxMap = []
        for i, utter in enumerate(self.dataX):
            for j in range(utter.shape[0]):
                self.idxMap.append((i, j)) # frame index, each frame has dim 40
        
    def __getitem__(self, idx):
        i, j = self.idxMap[idx]
        withContext = self.dataX[i].take(range(j - self.k, j + self.k + 1), mode='clip', axis=0).flatten()
        x = torch.Tensor(withContext).float()
        y = self.dataY[i][j] if self.dataY is not None else -1
        return x, y
    
    def __len__(self):
        return len(self.idxMap)

In [ ]:

#num_workers = 8 if cuda else 0 
num_workers = 0
# Training
train_dataset = MyDataset(train_data, hyper['kcontext'])

train_loader_args = dict(shuffle=True, batch_size=hyper['batchSize'], num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader = data.DataLoader(train_dataset, **train_loader_args,drop_last=True)

#Validation
val_dataset = MyDataset(dev_data, hyper['kcontext'])

val_loader_args = dict(shuffle=True, batch_size=hyper['batchSize'], num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
val_loader = data.DataLoader(val_dataset, **train_loader_args)

# Testing
test_dataset = MyDataset(test_data, hyper['kcontext'])

test_loader_args = dict(shuffle=False, batch_size=hyper['batchSize'], num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
test_loader = data.DataLoader(test_dataset, **test_loader_args)


In [ ]:
class Simple_MLP(nn.Module):
    def __init__(self, size_list):
        super(Simple_MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            layers.append(nn.BatchNorm1d(size_list[i+1]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.5))
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)
        #self.layers = nn.ModuleList(layers)

    def forward(self, x):
        return self.net(x)

In [ ]:
def one_epoch(epoch, net, loader, optimizer):
    net.train()
    running_loss = 0.0
    n = 0
    correct = 0
    total = 0
    start_time = time.time()
    for i, data in enumerate(loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        inupts = inputs.squeeze(0)
        outputs = net(inputs)
        loss = F.cross_entropy(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(f"\nFinished batch {i+1}  \t Timestamp: {time.time() - start_time}")
        if i % 5000 == 4999:
            print(f"\nFinished batch {i+1}  \t Timestamp: {time.time() - start_time}")
        #TODO: Here
        #if i % 10 == 9:
        #    print(f"\nFinished batch {i}  \t Timestamp: {time.time() - start_time}")
        #print(f"\nFinished batch {i}  \t Timestamp: {time.time() - start_time}")
        # print statistics
    avg_loss = running_loss / total
    acc = correct / total *100
    end_time = time.time()
    #print(f'\n\n++++++++++++++Epoch done++++++++++++\n')
    #print('Training Loss: %.8f \t Time: {%.3f}s\n\n'%(avg_loss, end_time - start_time))
    return avg_loss, acc

def train_step(net, loader, dev_loader, optimizer, scheduler, epochs):
    net.train()
    train_losses = []
    valid_losses = []
    valid_acc = []
    for epoch in range(epochs):  # loop over the dataset multiple times
        avg_loss_t, acc_t = one_epoch(epoch, net, loader, optimizer)
        avg_loss_v, acc_v = infer(net, dev_loader)
        #scheduler.step() # StepLR
        scheduler.step(avg_loss_v)
        print(f'\n\n*************\n')
        print('Epoch [%d], loss: %.8f, acc: %.4f' %
                (epoch + 1, avg_loss_t, acc_t))
        print('[valid] loss: %.8f, acc: %.4f\n\n' % (avg_loss_v, acc_v))
        #print("lr: {}".format(optimizer.param_groups[0]['lr']))
        '''
        if epoch % 5 == 4:
            print(f'\n\n*************\n')
            print('Epoch [%d], loss: %.8f, acc: %.4f' %
                  (epoch + 1, avg_loss_t, acc_t))
            print('[valid] loss: %.8f, acc: %.4f\n\n' % (avg_loss_v, acc_v))
            #print("lr: {}".format(optimizer.param_groups[0]['lr']))
        '''  
        train_losses.append(avg_loss_t)
        valid_losses.append(avg_loss_v)
        valid_acc.append(acc_v)
        print('\n','='*20)
        print("*** Saving Checkpoint ***\n")
        #path = "{}Model_Epoch_{}_v1.txt".format(hyper['weightDirName'], str(epoch))
        path = "gdrive/MyDrive/Bibek/"+"Model_"+str(epoch) #colab
        torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_losses,
            'dev_loss':valid_losses,
            'dev_acc': valid_acc
        }, path)
        
    return train_losses, valid_losses,valid_acc

# Xavier
def init_xavier(m):
    if type(m) == nn.Linear:
        fan_in = m.weight.size()[1]
        fan_out = m.weight.size()[0]
        std = np.sqrt(2.0 / (fan_in + fan_out))
        m.weight.data.normal_(0,std)

def train(net, loader, dev_loader, optimizer, epochs):
    net.train()
    train_losses = []
    valid_losses = []
    valid_acc = []
    for epoch in range(epochs):  # loop over the dataset multiple times
        avg_loss_t, acc_t = one_epoch(epoch, net, loader, optimizer)
        avg_loss_v, acc_v = infer(net, dev_loader)
        #scheduler.step()
        print(f'\n\n*************\n')
        print('Epoch [%d], loss: %.8f, acc: %.4f' %
                (epoch + 1, avg_loss_t, acc_t))
        print('[valid] loss: %.8f, acc: %.4f\n\n' % (avg_loss_v, acc_v))
        #print("lr: {}".format(optimizer.param_groups[0]['lr']))
        '''
        if epoch % 5 == 4:
            print(f'\n\n*************\n')
            print('Epoch [%d], loss: %.8f, acc: %.4f' %
                  (epoch + 1, avg_loss_t, acc_t))
            print('[valid] loss: %.8f, acc: %.4f\n\n' % (avg_loss_v, acc_v))
            #print("lr: {}".format(optimizer.param_groups[0]['lr']))
        '''  
        train_losses.append(avg_loss_t)
        valid_losses.append(avg_loss_v)
        valid_acc.append(acc_v)
        print('\n','='*20)
        print("*** Saving Checkpoint ***\n")
        #path = "{}optimContDecreaseCont_Epoch_{}_v1.txt".format(hyper['weightDirName'], str(epoch))
        path = "gdrive/MyDrive/Bibek/"+"Model_"+str(epoch) #colab
        torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_losses,
            'dev_loss':valid_losses,
            'dev_acc': valid_acc
        }, path)
        
    return train_losses, valid_losses, valid_acc

def infer(net, loader):
    net.eval()
    running_loss = 0.0
    n = 0
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():
        for i, data in enumerate(loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            # forward + backward + optimize
            outputs = net(inputs)
            loss = F.cross_entropy(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            #if i % 1000 == 999:
            #    print(f"\n(Eval)Finished batch{i}  \t Timestamp: {time.time() - start_time}")

    acc = correct / total * 100
    avg_loss = running_loss / total
    end_time = time.time()
    return avg_loss, acc

def predictLabels(model, test_loader, device):
    model.eval()
    
    res = np.array([])
    for batch_idx, (data, target) in enumerate(test_loader):
        data = data.to(device)
        target = target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, dim=1)
        res = np.concatenate((res, predicted.cpu().numpy().reshape(-1)))
    return res

In [ ]:
def main(hyper):
    """Main function to run everything with
    Args: 

        hyper(dict): Contains hyperparameter to tweak the model
    """
    input_size = (2 * hyper["kcontext"] + 1) * 13
    output_size = 346
    model = Simple_MLP([input_size] + hyper["hiddenDims"] + [output_size])
    
    #For loading the model
    #checkpoint = torch.load(hyper["checkpointPath"])
    #model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    #Initialize all the weights of linear layer
    model.apply(init_xavier)
    AdamOptimizer = torch.optim.Adam(model.parameters(),lr=hyper["lr"])
    #Lr scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(AdamOptimizer,mode='min',patience=3)
    #scheduler = torch.optim.lr_scheduler.StepLR(AdamOptimizer, step_size=33, gamma=0.5)
    print(model)

    train_losses, valid_losses, valid_acc = train_step(model, train_loader, val_loader, AdamOptimizer,scheduler,  hyper['nEpochs'])
    print(f'\n\n$$$$$$$$$Finished$$$$$$$$$$$$$\n\n')
    #train_losses, valid_losses = train(model, train_loader, val_loader, AdamOptimizer, hyper['nEpochs'])


    # Pridict label for test dataset
    labels = predictLabels(model, test_loader, device)
    np.save(hyper["testLabelPath"], labels)
    labels = list(map(int, labels))
    idxs = np.array(list(range(len(labels))))
    labels = np.array(labels)
    df = pd.DataFrame({"id" : idxs, "label" : labels})
    df.to_csv(hyper["testLabelCSVfn"], index=False)

In [ ]:
if __name__ == '__main__':
    main(hyper)

Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=325, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=1024, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=1024, out_features=512, bias=True)
    (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.5, inplace=False)
    (16): Linear(in_features=512, out_features=346, bias=True)
  )
)

Finished batch 5000  	 Timestamp: 59.27410840988159

F